In [20]:
from transformers import (
    AutoTokenizer, AutoModelForMaskedLM, 
)

# import nlpaug.augmenter.char as nac
# import nlpaug.augmenter.word as naw
# import nlpaug.augmenter.sentence as nas
# import nlpaug.flow as nafc
# from nlpaug.util import Action

# from googletrans import Translator
# import translators as ts

import re, math, random, json
from copy import deepcopy
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
import pandas as pd
from tqdm import tqdm
from collections import Counter

from module.preprocess import decorate_form, decorate_acd_pair, decorate_asc_pair, decorate_acd_pair_split, decorate_asc_pair_split
from module.utils import count_tags, make_token_classification_pair, remove_props, get_filter
import demoji

# from cleantext import clean
# from pykospacing import Spacing
# from hanspell import spell_checker

# Load Raw Data

In [21]:
train_json = './dataset/nikluge-sa-2022-train.jsonl'
dev_json = './dataset/nikluge-sa-2022-dev.jsonl'
test_json = './dataset/nikluge-sa-2022-test.jsonl'

train = pd.read_json(train_json, lines=True)
dev = pd.read_json(dev_json, lines=True)
test = pd.read_json(test_json, lines=True)

train = train.drop(2319)
dev = dev.drop(1692)

# Declare Stuff to use

In [22]:
### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'

]
more_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']

tf_id_to_name = ['True', 'False']
tf_name_to_id = {tf_id_to_name[i]: i for i in range(len(tf_id_to_name))}

polarity_id_to_name = ['positive', 'negative', 'neutral']
polarity_name_to_id = {polarity_id_to_name[i]: i for i in range(len(polarity_id_to_name))}

# Remove annotations without a target

In [23]:
total = pd.concat([train, dev])

In [24]:
### Remove
for idx, row in total.iterrows():
    for idx in range(len(row.annotation)):
        if row.annotation[idx][1][0] == None:
            row.annotation[idx] = []
    row.annotation = [el for el in row.annotation if el != []]

In [25]:
### Check
for idx, row in total.iterrows():
    for annotation in row.annotation:
        if annotation[1][0] == None:
            print(row)        

# Drop rows without a single annotation

In [26]:
total['checker'] = total.annotation.apply(bool)
total = total[total.checker == True].copy()

In [27]:
total['stratified'] = total.annotation.apply(lambda x: x[0][0])

In [28]:
total[total.id =='nikluge-sa-2022-train-02900']

,id,sentence_form,annotation,checker,stratified
2899,nikluge-sa-2022-train-02900,제품만족도 94% 제품이라 믿고 사용해본 #산다화클렌징오일,"[[본품#인지도, [산다화클렌징오일, 24, 32], positive]]",True,본품#인지도


In [29]:
count_tags(total, entity_property_pair)

tags found:  3475
tag set of df:  22
tag set of offered:  25
difference:  {'브랜드#디자인', '본품#가격', '제품 전체#다양성'}
본품#품질	1110
제품 전체#일반	788
본품#일반	454
제품 전체#품질	293
제품 전체#디자인	154
본품#편의성	104
패키지/구성품#디자인	97
브랜드#일반	94
제품 전체#편의성	87
제품 전체#인지도	78
패키지/구성품#편의성	55
패키지/구성품#일반	41
제품 전체#가격	28
본품#디자인	20
본품#다양성	20
패키지/구성품#품질	18
브랜드#품질	15
브랜드#인지도	12
패키지/구성품#다양성	3
브랜드#가격	2
본품#인지도	1
패키지/구성품#가격	1


In [30]:
filter = [x for x in entity_property_pair if x not in ['본품#인지도', '패키지/구성품#가격']]
total = remove_props(total, filter)
count_tags(total, entity_property_pair)

tags found:  3473
tag set of df:  20
tag set of offered:  25
difference:  {'패키지/구성품#가격', '본품#가격', '브랜드#디자인', '본품#인지도', '제품 전체#다양성'}
본품#품질	1110
제품 전체#일반	788
본품#일반	454
제품 전체#품질	293
제품 전체#디자인	154
본품#편의성	104
패키지/구성품#디자인	97
브랜드#일반	94
제품 전체#편의성	87
제품 전체#인지도	78
패키지/구성품#편의성	55
패키지/구성품#일반	41
제품 전체#가격	28
본품#디자인	20
본품#다양성	20
패키지/구성품#품질	18
브랜드#품질	15
브랜드#인지도	12
패키지/구성품#다양성	3
브랜드#가격	2


In [45]:
train, dev, _, _ = train_test_split(total, total['stratified'], test_size=0.2, random_state=42,  stratify=total['stratified'])
train.reset_index(inplace=True, drop=True)
dev.reset_index(inplace=True, drop=True)

In [46]:
count_tags(train, entity_property_pair)

tags found:  2788
tag set of df:  20
tag set of offered:  25
difference:  {'패키지/구성품#가격', '본품#가격', '브랜드#디자인', '본품#인지도', '제품 전체#다양성'}
본품#품질	893
제품 전체#일반	629
본품#일반	364
제품 전체#품질	237
제품 전체#디자인	124
본품#편의성	84
패키지/구성품#디자인	79
브랜드#일반	75
제품 전체#편의성	70
제품 전체#인지도	62
패키지/구성품#편의성	44
패키지/구성품#일반	34
제품 전체#가격	21
본품#다양성	16
본품#디자인	16
패키지/구성품#품질	14
브랜드#품질	12
브랜드#인지도	10
브랜드#가격	2
패키지/구성품#다양성	2


In [47]:
count_tags(dev, entity_property_pair)

tags found:  685
tag set of df:  19
tag set of offered:  25
difference:  {'패키지/구성품#가격', '본품#가격', '브랜드#가격', '브랜드#디자인', '본품#인지도', '제품 전체#다양성'}
본품#품질	217
제품 전체#일반	159
본품#일반	90
제품 전체#품질	56
제품 전체#디자인	30
본품#편의성	20
브랜드#일반	19
패키지/구성품#디자인	18
제품 전체#편의성	17
제품 전체#인지도	16
패키지/구성품#편의성	11
제품 전체#가격	7
패키지/구성품#일반	7
본품#다양성	4
본품#디자인	4
패키지/구성품#품질	4
브랜드#품질	3
브랜드#인지도	2
패키지/구성품#다양성	1


# Generate token classification pairs

In [ ]:
### test a sample
train['checker'] = train.annotation.apply(len)
original_input = train[train.checker > 1].iloc[2].sentence_form
annotations = train[train.checker > 1].iloc[2].annotation
original_input, annotations

In [48]:
def generate_token_classification_data(df):
    split_inputs, split_labels = [], []
    for original_input, annotations in zip(df.sentence_form, df.annotation):
        split_input, split_label = make_token_classification_pair(original_input, annotations)
        split_inputs.append(split_input)
        split_labels.append(split_label)
    return split_inputs, split_labels

train_split_inputs, train_split_labels = generate_token_classification_data(train)
dev_split_inputs, dev_split_labels = generate_token_classification_data(dev)

In [50]:
train['split_form'], train['split_label'] = train_split_inputs, train_split_labels
dev['split_form'], dev['split_label'] = dev_split_inputs, dev_split_labels

In [51]:
train

,id,sentence_form,annotation,checker,stratified,split_form,split_label
0,nikluge-sa-2022-dev-00536,화장은 지우는게 더 중요하니까 모두 화이트리로 깨끗하게 클렌징합시댜요,"[[제품 전체#일반, [화이트리, 20, 24], positive]]",True,제품 전체#일반,"[화장은 지우는게 더 중요하니까 모두 , 화이트리, 로 깨끗하게 클렌징합시댜요]","[0, 1, 0]"
1,nikluge-sa-2022-train-01485,100% 유기농 순면시트로 피부를 진정시켜주고 특허받은 안티세범P성분이 모공축소에 ...,"[[본품#품질, [100% 유기농 순면시트, 0, 13], positive], [본...",True,본품#품질,"[100% 유기농 순면시트, 로 피부를 진정시켜주고 특허받은 , 안티세범P성분, 이...","[1, 0, 1, 0]"
2,nikluge-sa-2022-dev-00507,휴대도 간편한 피치풋밤~,"[[제품 전체#편의성, [피치풋밤, 8, 12], positive]]",True,제품 전체#편의성,"[휴대도 간편한 , 피치풋밤, ~]","[0, 1, 0]"
3,nikluge-sa-2022-train-01889,매일 매일 발라도 부담없는 #데일리대용량앰플,"[[본품#일반, [데일리대용량앰플, 16, 24], positive]]",True,본품#일반,"[매일 매일 발라도 부담없는 #, 데일리대용량앰플]","[0, 1]"
4,nikluge-sa-2022-dev-01502,#다이슨 바람 느껴보니 왜 유명한줄 알겠다..,"[[제품 전체#인지도, [다이슨, 1, 4], positive]]",True,제품 전체#인지도,"[#, 다이슨, 바람 느껴보니 왜 유명한줄 알겠다..]","[0, 1, 0]"
...,...,...,...,...,...,...,...
2575,nikluge-sa-2022-dev-02278,"동안피부의 6가지 조건인 윤기, 촉촉함, 탄력, 주름, 피부톤, 피부결을 동시에 케...","[[본품#품질, [에이지투웨니스 #시그니처퍼스트에센스, 55, 74], positi...",True,본품#품질,"[동안피부의 6가지 조건인 윤기, 촉촉함, 탄력, 주름, 피부톤, 피부결을 동시에 ...","[0, 1, 0]"
2576,nikluge-sa-2022-train-00632,닥터홍 홍유 필오프팩은 필오프팩 제품군 중 겔이 10분 이내로 굳는 제일 빠른 제품이에용,"[[제품 전체#일반, [닥터홍 홍유 필오프팩, 0, 11], positive]]",True,제품 전체#일반,"[닥터홍 홍유 필오프팩, 은 필오프팩 제품군 중 겔이 10분 이내로 굳는 제일 빠른...","[1, 0]"
2577,nikluge-sa-2022-dev-02351,폴리스캐리어 24형 덕분에 신혼여행 잘 갔다왔습니당 ㅎㅅㅎ!,"[[제품 전체#일반, [폴리스캐리어 24형, 0, 10], positive]]",True,제품 전체#일반,"[폴리스캐리어 24형, 덕분에 신혼여행 잘 갔다왔습니당 ㅎㅅㅎ!]","[1, 0]"
2578,nikluge-sa-2022-dev-00727,#라이프플랑크톤에센스 벌써 3병째..,"[[제품 전체#일반, [라이프플랑크톤에센스, 1, 11], positive]]",True,제품 전체#일반,"[#, 라이프플랑크톤에센스, 벌써 3병째..]","[0, 1, 0]"


In [55]:
from transformers import (
    AutoConfig, ElectraTokenizer, ElectraForTokenClassification, 
    DataCollatorForTokenClassification,
    TrainingArguments, Trainer,
)

In [56]:
target_tagger_labels = ['Other', 'Target']
labels = target_tagger_labels
label2id = {k: i for i, k in enumerate(labels)}
id2label = {i: k for i, k in enumerate(labels)}
num_labels = len(labels)

In [59]:
model_checkpoint = 'monologg/koelectra-base-v3-discriminator'
tokenizer = ElectraTokenizer.from_pretrained(model_checkpoint)
model = ElectraForTokenClassification.from_pretrained(
    model_checkpoint, label2id=label2id, id2label=id2label, num_labels=num_labels
)
model.config.label2id, model.config.id2label, model.num_labels

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier

({'Other': 0, 'Target': 1}, {0: 'Other', 1: 'Target'}, 2)

In [83]:
split_form = ['화장은 지우는게 더 중요하니까 모두 ', '화이트리', '로 깨끗하게 클렌징합시댜요']
label = [0, 1, 0]

In [84]:
for idx in range(len(split_form)):
    tokens = tokenizer.tokenize(split_form[idx])
    label[idx] = [label[idx] for _ in range(len(tokens))]
    split_form[idx] = tokens

split_form = [x for el in split_form for x in el]
label = [x for el in label for x in el]

# CHECK POINT

In [85]:
split_form, label

(['화장',
  '##은',
  '지우',
  '##는',
  '##게',
  '더',
  '중요',
  '##하니',
  '##까',
  '모두',
  '화이트',
  '##리',
  '로',
  '깨끗',
  '##하',
  '##게',
  '클렌징',
  '##합',
  '##시',
  '##댜',
  '##요'],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [81]:
for x, y in zip(split_form, label):
    print(len(x), len(y))
    print()

10 10

2 2

9 9



In [ ]:
# count = 0
# for x, y in zip(split_inputs, split_labels):
#     print(x)
#     print(y)
#     print()
#     count += 1
#     if count == 99:
#         break

# Count

In [ ]:
count_tags(train, entity_property_pair)

# Filter entity_property_pair and Drop rows accordingly

In [ ]:
FILTER_MODE = False

In [ ]:
if FILTER_MODE == True:
    filter = get_filter()
    train = remove_props(train, filter)
    dev = remove_props(dev, filter)
len(train), len(dev)

# Preprocess

## Cleansing

### Before

In [ ]:
# for el in train.sample(n=5).sentence_form:
#     print(el)

In [ ]:
# train.sentence_form = train.sentence_form.apply(preprocess)
# dev.sentence_form = dev.sentence_form.apply(preprocess)
# test.sentence_form = test.sentence_form.apply(preprocess)
# total = pd.concat([train, dev])

### Test

In [ ]:
# case = total.sentence_form.str.contains('r[^A-Za-z0-9가-힣\s]+', case=False, flags=0, na=None, regex=True)
# for e in total[case].sentence_form:
#     print(e)

### After

In [ ]:
# for i, row in total[['id', 'sentence_form']].sample(n=5).iterrows():
#     print(row.id, '\t', row.sentence_form)

In [ ]:
# total['check'] = total.sentence_form.str.find('OO')
# for row in total[total.check > -1].sentence_form:
#     print(row)
#     break

In [ ]:
# total

## Reformat

In [ ]:
len(entity_property_pair)

In [ ]:
decorate_form, decorate_acd_pair, decorate_asc_pair, decorate_acd_pair_split, decorate_asc_pair_split

In [ ]:
def reformat(df):
    ep =[]
    p = []
    for index, row in df.iterrows():
        utterance = row.sentence_form
        id = row.id
        
        form = utterance
        # form = decorate_form(utterance)

        for pair in entity_property_pair:
            isPairInOpinion = False
            if pd.isna(utterance):
                break
            for annotation in row.annotation:
                entity_property = annotation[0]
                sentiment = annotation[2]
                if entity_property == pair:
                    
                    acd_pair = entity_property
                    # acd_pair = decorate_acd_pair(entity_property)
                    # acd_pair = decorate_acd_pair_split(entity_property)
                    
                    ep_append = [id, form, acd_pair, tf_name_to_id['True']]
                    ep.append(ep_append)
                    p.append([id, utterance, entity_property, sentiment])
                    isPairInOpinion = True
                    break
            if isPairInOpinion is False:
                
                acd_pair = pair
                # acd_pair = decorate_acd_pair(pair)
                # acd_pair = decorate_acd_pair_split(pair)
                
                ep_append = [id, form, acd_pair, tf_name_to_id['False']]
                ep.append(ep_append)
    return ep, p

In [ ]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.form, row.pair, row.sentiment
        
        form = row.form
        # form = decorate_form(row.form)
        
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:

                asc_pair = '#'.join([row.pair, row.sentiment])
                # asc_pair = decorate_asc_pair(row.pair, row.sentiment)
                # asc_pair = decorate_asc_pair_split(row.pair, row.sentiment)

                p_binary_append = [row.id, form, asc_pair, tf_name_to_id['True']]
                p_binary.append(p_binary_append)
            else:

                asc_pair = '#'.join([row.pair, sentiment])
                # asc_pair = decorate_asc_pair(row.pair, sentiment)
                # asc_pair = decorate_asc_pair_split(row.pair, sentiment)

                p_binary_append = [row.id, form, asc_pair, tf_name_to_id['False']]
                p_binary.append(p_binary_append)
    return p_binary

In [ ]:
len(train), len(dev)

In [ ]:
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'

]

In [ ]:
ep_train, p_train = reformat(train)
ep_dev, p_dev = reformat(dev)

ep_train = pd.DataFrame(ep_train, columns=['id', 'form', 'pair', 'labels'])
ep_dev = pd.DataFrame(ep_dev, columns=['id', 'form', 'pair', 'labels'])

p_train = pd.DataFrame(p_train, columns=['id', 'form', 'pair', 'sentiment'])
p_dev = pd.DataFrame(p_dev, columns=['id', 'form', 'pair', 'sentiment'])

len(ep_train), len(ep_dev), len(p_train), len(p_dev)

In [ ]:
p_binary_train = reformat_p_binary(p_train)
p_binary_train = pd.DataFrame(p_binary_train, columns=['id', 'form', 'pair', 'labels'])

p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'form', 'pair', 'labels'])

len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev)

In [ ]:
# ep_train.sort_values(['id', 'labels'], inplace=True)
# ep_dev.sort_values(['id', 'labels'], inplace=True)
# p_binary_train.sort_values(['id', 'labels'], inplace=True, ascending=[True, True])
# p_binary_dev.sort_values(['id', 'labels'], inplace=True, ascending=[True, True])

### Counting

In [ ]:
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))
ep_train = ep_train.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_train = p_train.drop_duplicates()
p_dev = p_dev.drop_duplicates()
p_binary_train = p_binary_train.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
print('\nafter drop_duplicates\n')
print('binary_multi: ', end=''), print(len(ep_train), len(ep_dev), len(p_train), len(p_dev))
print('binary_binary: ', end=''), print(len(ep_train), len(ep_dev), len(p_binary_train), len(p_binary_dev))

### Validate Here

In [ ]:
df = ep_train
for idx, row in df.iterrows():
    print(row.id, '\n',
          row.form, '\n',
          row.pair, '\n',
          row.labels,  '\n',)
    if idx == 49:
        break

### Save

In [ ]:
DATA_V = 'uncleaned_v6'
save_path = f'./dataset/{DATA_V}'
print(save_path)

In [ ]:
!mkdir -p {save_path}

train.to_csv(f'{save_path}/raw_train.csv', index=False)
dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
test.to_csv(f'{save_path}/raw_test.csv', index=False)

ep_train.to_csv(f'{save_path}/ce_train.csv', index=False)
ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)

p_binary_train.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
p_binary_dev.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

### Additional Length Test If Needed

In [ ]:
# ep_train, ep_dev, p_binary_train, p_binary_dev

In [ ]:
model_checkpoint = 'snunlp/KR-ELECTRA-discriminator'

train_path = f'./dataset/{DATA_V}/raw_train.csv'
dev_path = f'./dataset/{DATA_V}/raw_dev.csv'
test_path = f'./dataset/{DATA_V}/raw_test.csv'
train = pd.read_csv(train_path)
dev = pd.read_csv(dev_path)
test = pd.read_csv(test_path)

### new
entity_property_pair = [
    '본품#가격', '본품#다양성', '본품#디자인', '본품#인지도', '본품#일반', '본품#편의성', '본품#품질',
    '브랜드#가격', '브랜드#디자인', '브랜드#인지도', '브랜드#일반', '브랜드#품질',
    '제품 전체#가격', '제품 전체#다양성', '제품 전체#디자인', '제품 전체#인지도', '제품 전체#일반', '제품 전체#편의성', '제품 전체#품질',
    '패키지/구성품#가격', '패키지/구성품#다양성', '패키지/구성품#디자인', '패키지/구성품#일반', '패키지/구성품#편의성', '패키지/구성품#품질'
]
special_tokens = ['&name&', '&affiliation&', '&social-security-num&', '&tel-num&', '&card-num&', '&bank-account&', '&num&', '&online-account&']
emojis = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))
ep_labels = pd.Series(entity_property_pair, name='sentence_form', copy=True)

tokens2add = special_tokens + emojis

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
print(len(tokenizer))
tokenizer_train_data = pd.concat([train.sentence_form, dev.sentence_form, test.sentence_form], ignore_index=True, verify_integrity=True).to_frame().drop_duplicates()
tokenizer_train_data = tokenizer_train_data.sentence_form.to_list()
new_tokenizer = tokenizer.train_new_from_iterator(tokenizer_train_data, vocab_size=1)
new_tokens = set(list(new_tokenizer.vocab.keys()) + tokens2add) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
print(len(new_tokenizer))
print(len(tokenizer))
# model.resize_token_embeddings(len(tokenizer))

In [ ]:
ep_train, ep_dev, p_binary_train, p_binary_dev
len_counter = []
for df in [ep_train, ep_dev, p_binary_train, p_binary_dev]:
    for idx, row in df.iterrows():
        len_counter.append(len(tokenizer(row["form"], row["pair"], truncation=True).input_ids))

In [ ]:
max(len_counter)

### done here.

## Save Files

In [ ]:
# save_path = './dataset/cleaned_v1'

# train.to_csv(f'{save_path}/raw_train.csv', index=False)
# dev.to_csv(f'{save_path}/raw_dev.csv', index=False)
# test.to_csv(f'{save_path}/raw_test.csv', index=False)

# ep_train.to_csv(f'{save_path}/ce_train.csv', index=False)
# ep_dev.to_csv(f'{save_path}/ce_dev.csv', index=False)
# p_train.to_csv(f'{save_path}/pc_train.csv', index=False)
# p_dev.to_csv(f'{save_path}/pc_dev.csv', index=False)
# p_binary_train.to_csv(f'{save_path}/pc_binary_train.csv', index=False)
# p_binary_dev.to_csv(f'{save_path}/pc_binary_dev.csv', index=False)

# ASC Augmentation

In [ ]:
model_checkpoint = '/content/drive/MyDrive/aspect_based_sentiment_analysis/base_model/klue_roberta_base/v2/klue_roberta_base_mlm/checkpoint-19860'

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForMaskedLM.from_pretrained(model_checkpoint)

In [ ]:
sTokens = tokenizer.all_special_tokens

def delTokens(sent):
    sent = sent.split(' ')
    temp = []
    for e in sent:
        if e not in sTokens:
            temp.append(e)
    return ' '.join(temp)

In [ ]:
positive, negative, neutral = p_train[p_train.sentiment == 'positive'], p_train[p_train.sentiment == 'negative'], p_train[p_train.sentiment == 'neutral']

In [ ]:
len(positive), len(negative), len(neutral)

In [ ]:
(58 * 3) * 4 * 3, (95 * 3) * 4 * 2 # bt ri rr

Back Translation / Random Insertion / Random Replacement / Random Swap / Random Deletion

In [ ]:
def backTrans(text):
    aug1 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='en')
    aug1 = ts.papago(aug1, sleep_seconds=5, from_language='en', to_language='ko')

    aug2 = ts.papago(text, sleep_seconds=5, from_language='ko', to_language='ja')
    aug2 = ts.papago(aug2, sleep_seconds=5, from_language='ja', to_language='ko')

    return [aug1, aug2]

def randomInsert(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomReplace(num, sample, device):
    aug = naw.ContextualWordEmbsAug(
        model_path=model_checkpoint, action="insert", model_type='bert', top_k=5, aug_p=0.3, aug_min=1, aug_max=1, device=device)

    aug_result = aug.augment(sample, n=num, num_thread=12)
    aug_result = list(map(delTokens, aug_result))
    aug_result = list(set(aug_result))
    return aug_result

def randomSwap(num, sample):
    aug = naw.RandomWordAug(action='swap', aug_min=1, aug_max=1, aug_p=0.3)    
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

def randomSplit(num, sample):
    aug = naw.SplitAug(aug_min=1, aug_max=1, aug_p=0.3, min_char=3)
    aug_result = aug.augment(sample, n=num, num_thread=2)
    aug_result = list(set(aug_result))
    return aug_result

In [ ]:
(58 * 3) * 5 * 4, (95 * 3) * 4 * 3 # bt ri rr

In [ ]:
def backtransRoutine(data2augment, output_path):
    print('back translation started.')
    temp = []
    for row in data2augment:
        augs = backTrans(row[1])
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(len(f'back translation finished.\ncurrent count: {len(data2augment)}'))

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
import os

def edaRoutine(data2augment, ri, rr, output_path):
    print(f'current count: {len(data2augment)}')
    print('random insertion started.')
    temp = []
    for row in data2augment:
        augs = randomInsert(ri, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random insertion finished.\ncurrent count: {len(data2augment)}')

    print('random replacement started.')
    temp = []
    for row in data2augment:
        augs = randomReplace(rr, row[1], 'cuda')
        for aug in augs:
            if aug != '' and aug != row[1]:
                new = [row[0], aug, row[2], row[3]]
                if new not in data2augment:
                    temp.append(new)
    data2augment.extend(temp)
    print(f'random replacement finished.\ncurrent count: {len(data2augment)}')

    print('random swap and split started.')
    while len(data2augment) < len(positive):
        temp = []
        k = random.randrange(len(negative))
        id, text, entity, sentiment = data2augment[k]

        selector = random.randint(0,1)
        if selector == 0:
            augs = randomSwap(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        else:
            augs = randomSplit(1, text)
            for aug in augs:
                if aug != '' and aug != text:
                    new = [id, aug, entity, sentiment]
                    if new not in data2augment:
                        temp.append(new)
            data2augment.extend(temp)
        if len(data2augment)%25 == 0:
            print(f'random swap and split in progress.\ncurrent count: {len(data2augment)}')

    print(f'whole augmentation routine finished.\ntotal count: {len(data2augment)}')

    data_aug = pd.DataFrame(data2augment, columns=['id', 'sentence_form', 'entity_property', 'sentiment'])
    data_aug.to_csv(f'{output_path}', index=False)

    return data_aug

In [ ]:
### negative
# # back translation

# data2augment = negative.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_bt.csv'
output_path = os.path.join(output_folder, output_file)

# negative_bt = backtransRoutine(data2augment, output_path)
negative_bt = pd.read_csv(output_path)
negative_bt = negative_bt.values.tolist()
# RI / RR

ri = 4 # times - 1
rr = 3 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'negative_aug.csv'
output_path = os.path.join(output_folder, output_file)

# negative_aug = edaRoutine(negative_bt, ri, rr, output_path)
negative_aug = pd.read_csv(output_path)

In [ ]:
negative_aug
negative_aug = negative_aug.drop_duplicates()

In [ ]:
# negative_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# negative_aug.sort_values('id').head(50).sentence_form

In [ ]:
### neutral
# back translation

# data2augment = neutral.values.tolist()

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_bt.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_bt = backtransRoutine(data2augment, output_path)
neutral_bt = pd.read_csv(output_path)
neutral_bt = neutral_bt.values.tolist()

# RI / RR

ri = 3 # times - 1
rr = 2 # times - 1

output_folder = '/content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11'
output_file = 'neutral_aug.csv'
output_path = os.path.join(output_folder, output_file)

# neutral_aug = edaRoutine(neutral_bt, ri, rr, output_path)
neutral_aug = pd.read_csv(output_path)

In [ ]:
neutral_aug
neutral_aug = neutral_aug.drop_duplicates()

In [ ]:
# neutral_aug.sample(n=15, frac=None, replace=False, weights=None, random_state=None, axis=None, ignore_index=False)

In [ ]:
# neutral_aug.sort_values('id').head(50).sentence_form

In [ ]:
p_train_aug = pd.concat([positive, negative_aug, neutral_aug])

In [ ]:
def reformat_p_binary(df):
    p_binary = []
    for i, row in df.iterrows():
        row.id, row.sentence_form, row.entity_property, row.sentiment
        for sentiment in polarity_id_to_name:
            if sentiment == row.sentiment:
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, row.sentiment]), tf_name_to_id['True']])
            else: 
                p_binary.append([row.id, row.sentence_form, '#'.join([row.entity_property, sentiment]), tf_name_to_id['False']])
    return p_binary

In [ ]:
p_binary_train_aug = reformat_p_binary(p_train_aug)
p_binary_train_aug = pd.DataFrame(p_binary_train_aug, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_train_aug

In [ ]:
p_binary_dev = reformat_p_binary(p_dev)
p_binary_dev = pd.DataFrame(p_binary_dev, columns=['id', 'sentence_form', 'entity_property', 'labels'])

In [ ]:
p_binary_dev

# Counting

In [ ]:
len(ep_train), len(ep_dev), len(p_train), len(p_dev)

In [ ]:
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

In [ ]:
ep_train = ep_train.drop_duplicates()
p_binary_train_aug = p_binary_train_aug.drop_duplicates()
ep_dev = ep_dev.drop_duplicates()
p_binary_dev = p_binary_dev.drop_duplicates()
len(ep_train), len(ep_dev), len(p_binary_train_aug), len(p_binary_dev)

# Export

In [ ]:
%cd /content/drive/MyDrive/aspect_based_sentiment_analysis/data/v11

# train.to_csv('raw_train.csv', index=False)
# dev.to_csv('raw_dev.csv', index=False)
# test.to_csv('raw_test.csv', index=False)

ep_train.to_csv('ce_train.csv', index=False)
p_binary_train_aug.to_csv('pc_binary_train_aug.csv', index=False)
ep_dev.to_csv('ce_dev.csv', index=False)
p_binary_dev.to_csv('pc_binary_dev.csv', index=False)

In [ ]:
# emojis = pd.concat([ep_train.sentence_form, p_train.sentence_form, ep_dev.sentence_form, p_dev.sentence_form], ignore_index=True, verify_integrity=True).to_frame()
# emojis = list(set(demoji.findall(' '.join(emojis.sentence_form.to_list())).keys()))

In [ ]:
df = pd.read_csv('ce_train.csv')
df[df.id == 'nikluge-sa-2022-train-00065']

In [ ]:
df = pd.read_csv('ce_dev.csv')
df

In [ ]:
df = pd.read_csv('pc_binary_train_aug.csv')
df

In [ ]:
df = pd.read_csv('pc_binary_dev.csv')
df